 # 05_insights.ipynb — PROYECTO AURORA



 En este notebook NO exploramos más, sino que:

 - Resumimos los KPIs clave por plataforma.

 - Extraemos insights accionables de usuarios y monetización.

 - Miramos estacionalidad (trimestres) y geografía (Steam).

 - Analizamos la concentración de gasto (whales) con PlayFab.



 Este notebook sirve de puente entre el EDA técnico y:

 - La narrativa de negocio.

 - El dashboard final en Tableau.

In [24]:
import os
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

BASE_PATH = ".."
FINAL_PATH = os.path.join(BASE_PATH, "data", "final")
CLEAN_PATH = os.path.join(BASE_PATH, "data", "clean")
RAW_PATH = os.path.join(BASE_PATH, "data", "raw")

print("FINAL_PATH →", FINAL_PATH)


FINAL_PATH → ..\data\final


 ## 1. Carga de datos consolidados

In [25]:
# FACT maestro día x plataforma
fact_path = os.path.join(FINAL_PATH, "fact_diario.csv")
fact = pd.read_csv(fact_path)

fact["Fecha"] = pd.to_datetime(fact["Fecha"], errors="coerce")

# Por si acaso, aseguramos columnas derivadas (versión "amiga" de Pylance)
if "AñoMes" not in fact.columns:
    fact["AñoMes"] = pd.PeriodIndex(fact["Fecha"], freq="M").astype(str)

if "AñoTrimestre" not in fact.columns:
    fact["AñoTrimestre"] = pd.PeriodIndex(fact["Fecha"], freq="Q").astype(str)
if "DAU_roll7" not in fact.columns:
    fact = fact.sort_values("Fecha")
    fact["DAU_roll7"] = fact.groupby("Plataforma")["DAU"].transform(
        lambda x: x.rolling(7, min_periods=1).mean()
    )

if "Ingresos_roll7" not in fact.columns:
    fact = fact.sort_values("Fecha")
    fact["Ingresos_roll7"] = fact.groupby("Plataforma")["Ingresos_USD"].transform(
        lambda x: x.rolling(7, min_periods=1).mean()
    )

print("Shape FACT:", fact.shape)
print("Plataformas:", fact["Plataforma"].unique())
print("Rango fechas:", fact["Fecha"].min(), "→", fact["Fecha"].max())


Shape FACT: (2596, 23)
Plataformas: ['Canal A' 'Canal B']
Rango fechas: 2022-06-01 00:00:00 → 2025-10-31 00:00:00


 ## 2. KPIs globales por plataforma



 Queremos una foto rápida:

 - ¿Qué canal tiene más usuarios (DAU)?

 - ¿Quién genera más ingresos totales?

 - ¿Quién monetiza mejor por día?

 - ¿Cómo se comportan las descargas?

In [26]:
kpi = (
    fact.groupby("Plataforma")
    .agg(
        DAU_medio=("DAU", "mean"),
        DAU_total=("DAU", "sum"),
        Descargas_total=("Descargas", "sum"),
        Ingresos_total=("Ingresos_USD", "sum"),
        Dias=("Fecha", "nunique"),
    )
)

kpi["Ingresos_dia_medio"] = kpi["Ingresos_total"] / kpi["Dias"]
kpi["Descargas_dia_media"] = kpi["Descargas_total"] / kpi["Dias"]

display(kpi.round(2))

# Insight automático básico
def comparar(metric: str) -> str:
    """Devuelve una frase comparando Canal A vs Canal B en una métrica numérica."""
    try:
        a_raw = kpi.loc["Canal A", metric]
        b_raw = kpi.loc["Canal B", metric]
    except KeyError:
        return f"- {metric}: falta alguna plataforma en kpi."

    # Forzamos a float para que el tipado esté contento
    a = float(a_raw)
    b = float(b_raw)

    # Si son casi iguales, lo decimos y listo
    if abs(a - b) < 1e-6:
        return f"- {metric}: Canal A y Canal B tienen valores muy similares."

    ganador = "Canal A" if a > b else "Canal B"
    perdedor = "Canal B" if a > b else "Canal A"

    # Evitar división por cero
    base = b if ganador == "Canal A" else a
    if abs(base) < 1e-9:
        return f"- {metric}: {ganador} tiene valor > 0 y {perdedor} ≈ 0."

    ratio = (max(a, b) / base) - 1
    return (
        f"- {metric}: **{ganador}** supera a {perdedor} aproximadamente en "
        f"{ratio*100:.1f}%."
    )



print("\n📌 Comparativa automática de KPIs:")
for m in ["DAU_medio", "DAU_total", "Ingresos_total", "Ingresos_dia_medio", "Descargas_dia_media"]:
    print(comparar(m))


,DAU_medio,DAU_total,Descargas_total,Ingresos_total,Dias,Ingresos_dia_medio,Descargas_dia_media
Plataforma,,,,,,,
Canal A,1496.99,2016447,74508.0,1256988.89,1249,1006.40,59.65
Canal B,3241.53,4048675,213439.0,941352.55,1249,753.68,170.89



📌 Comparativa automática de KPIs:
- DAU_medio: **Canal B** supera a Canal A aproximadamente en 116.5%.
- DAU_total: **Canal B** supera a Canal A aproximadamente en 100.8%.
- Ingresos_total: **Canal A** supera a Canal B aproximadamente en 33.5%.
- Ingresos_dia_medio: **Canal A** supera a Canal B aproximadamente en 33.5%.
- Descargas_dia_media: **Canal B** supera a Canal A aproximadamente en 186.5%.


 ## 3. Relación usuarios ↔ ingresos



 Preguntas:

 - ¿Cuánto se parecen los patrones de DAU y de ingresos?

 - ¿La monetización sigue de cerca a la base de usuarios?



 Lo miramos con correlaciones y con una tabla muy simple.

In [27]:
# Correlaciones globales entre métricas clave
cols_core = ["DAU", "Descargas", "Ingresos_USD", "DAU_roll7", "Ingresos_roll7"]
corr_core = fact[cols_core].corr().round(3)

print("===== Correlaciones CORE (todas las plataformas) =====")
print(corr_core)

# Small summary: DAU vs ingresos
corr_dau_ing = corr_core.loc["DAU", "Ingresos_USD"]
print(f"\n🔎 Correlación DAU ↔ Ingresos_USD (global): {corr_dau_ing:.3f}")

# Ahora vemos por plataforma
for plataforma in fact["Plataforma"].unique():
    sub = fact[fact["Plataforma"] == plataforma]
    c = sub[["DAU", "Ingresos_USD"]].corr().iloc[0, 1]
    print(f"- {plataforma}: corr(DAU, Ingresos_USD) = {c:.3f}")


===== Correlaciones CORE (todas las plataformas) =====
                  DAU  Descargas  Ingresos_USD  DAU_roll7  Ingresos_roll7
DAU             1.000      0.219        -0.009      0.994          -0.041
Descargas       0.219      1.000         0.167      0.190           0.033
Ingresos_USD   -0.009      0.167         1.000     -0.050           0.438
DAU_roll7       0.994      0.190        -0.050      1.000          -0.058
Ingresos_roll7 -0.041      0.033         0.438     -0.058           1.000

🔎 Correlación DAU ↔ Ingresos_USD (global): -0.009
- Canal A: corr(DAU, Ingresos_USD) = 0.223
- Canal B: corr(DAU, Ingresos_USD) = 0.189


 ## 4. Estacionalidad y evolución temporal



 Aquí nos interesa:

 - ¿Qué canal crece más a lo largo del tiempo?

 - ¿Hay trimestres fuertes / flojos para cada canal?

 - ¿Quién domina en ingresos trimestrales?

In [28]:
# Ingresos por trimestre y plataforma
ing_trimestral = (
    fact.groupby(["AñoTrimestre", "Plataforma"])["Ingresos_USD"]
    .sum()
    .reset_index()
    .sort_values(["AñoTrimestre", "Plataforma"])
)

print("📌 Ingresos por trimestre (USD):")
display(ing_trimestral.head(20))

# Ratio de crecimiento: primeros 4 trimestres vs últimos 4
def resumen_crecimiento(df, plataforma):
    sub = df[df["Plataforma"] == plataforma].copy()
    sub = sub.sort_values("AñoTrimestre")
    if sub["AñoTrimestre"].nunique() < 4:
        return f"- {plataforma}: pocos trimestres para hablar de crecimiento."
    # Primeros 4 y últimos 4 (si hay)
    primeros = sub.head(4)["Ingresos_USD"].mean()
    ultimos = sub.tail(4)["Ingresos_USD"].mean()
    if np.isclose(primeros, 0):
        return f"- {plataforma}: ingresos iniciales casi nulos, difícil comparar."
    ratio = (ultimos / primeros) - 1
    return (
        f"- {plataforma}: los últimos 4 trimestres facturan de media "
        f"{ratio*100:.1f}% más (o menos, si negativo) que los primeros 4."
    )

print("\n📌 Crecimiento aproximado por canal (primeros 4 trimestres vs últimos 4):")
for plat in fact["Plataforma"].unique():
    print(resumen_crecimiento(ing_trimestral, plat))


📌 Ingresos por trimestre (USD):


,AñoTrimestre,Plataforma,Ingresos_USD
0,2022Q2,Canal A,32607.16
1,2022Q2,Canal B,22698.97
2,2022Q3,Canal A,91276.21
3,2022Q3,Canal B,78781.00
4,2022Q4,Canal A,80326.37
5,2022Q4,Canal B,58626.36
6,2023Q1,Canal A,90049.93
7,2023Q1,Canal B,54998.57
8,2023Q2,Canal A,87083.53
9,2023Q2,Canal B,71095.54



📌 Crecimiento aproximado por canal (primeros 4 trimestres vs últimos 4):
- Canal A: los últimos 4 trimestres facturan de media -1.7% más (o menos, si negativo) que los primeros 4.
- Canal B: los últimos 4 trimestres facturan de media -10.4% más (o menos, si negativo) que los primeros 4.


 ## 5. Monetización por canal



 - Canal A: métricas de ventas in-game (Ingresos_Brutos, Unidades, etc.)

 - Canal B: métricas de oro gastado y ARP*.



 Vamos a resumir:

 - Eficiencia de ingresos por DAU.

 - Relación entre DAU y métricas de monetización.

In [29]:
# KPIs básicos de monetización por plataforma
def resumen_monetizacion(df, plataforma):
    sub = df[df["Plataforma"] == plataforma].copy()
    dias = sub["Fecha"].nunique()

    total_dau = sub["DAU"].sum()
    total_ing = sub["Ingresos_USD"].sum()

    dau_medio = sub["DAU"].mean()
    ing_dia = total_ing / dias if dias > 0 else np.nan
    ing_por_dau = total_ing / total_dau if total_dau > 0 else np.nan

    res = {
        "Plataforma": plataforma,
        "Días": dias,
        "DAU_medio": dau_medio,
        "Ingresos_total": total_ing,
        "Ingresos_dia_medio": ing_dia,
        "Ingresos_por_DAU": ing_por_dau,
    }

    # Si hay columnas de ARPU/ARPDAU/Oro, añadimos estadísticas
    for col in ["Oro_Gastado_USD", "ARPDAU", "ARPU", "ARPPU"]:
        if col in sub.columns and sub[col].notna().sum() > 0:
            res[f"{col}_medio"] = sub[col].mean()
        else:
            res[f"{col}_medio"] = np.nan

    return res

res_list = [resumen_monetizacion(fact, plat) for plat in fact["Plataforma"].unique()]
df_monet_kpi = pd.DataFrame(res_list)
display(df_monet_kpi.round(3))


,Plataforma,Días,DAU_medio,Ingresos_total,Ingresos_dia_medio,Ingresos_por_DAU,Oro_Gastado_USD_medio,ARPDAU_medio,ARPU_medio,ARPPU_medio
0,Canal A,1249,1496.991,1256988.89,1006.396,0.623,NaN,NaN,NaN,NaN
1,Canal B,1249,3241.533,941352.55,753.685,0.233,753.685,23.836,0.197,2947.73


 ## 6. Correlaciones de monetización (detalle por canal)



 Aprovechamos el trabajo del EDA, pero aquí solo mostramos tablas y

 nos quedamos con la lectura más importante para negocio.

In [30]:
# Canal A
canal_a = fact[fact["Plataforma"] == "Canal A"].copy()
cols_a = [
    "DAU",
    "Descargas",
    "Ingresos_USD",
    "Ingresos_Brutos_USD",
    "Unidades_Vendidas",
    "Usuarios_Concurrentes_Pico",
    "DAU_roll7",
    "Ingresos_roll7",
]
corr_a = canal_a[cols_a].corr().round(3)

print("===== Correlaciones monetización — Canal A =====")
print(corr_a)

# Canal B
canal_b = fact[fact["Plataforma"] == "Canal B"].copy()
cols_b = [
    "DAU",
    "Descargas",
    "Ingresos_USD",
    "Oro_Gastado_USD",
    "ARPU",
    "ARPDAU",
    "ARPPU",
    "DAU_roll7",
    "Ingresos_roll7",
]
cols_b_validas = [c for c in cols_b if c in canal_b.columns]
corr_b = canal_b[cols_b_validas].corr().round(3)

print("\n===== Correlaciones monetización — Canal B =====")
print(corr_b)


===== Correlaciones monetización — Canal A =====
                              DAU  Descargas  Ingresos_USD  \
DAU                         1.000      0.126         0.223   
Descargas                   0.126      1.000         0.128   
Ingresos_USD                0.223      0.128         1.000   
Ingresos_Brutos_USD         0.225      0.128         1.000   
Unidades_Vendidas           0.215      0.149         0.910   
Usuarios_Concurrentes_Pico  0.540      0.026         0.202   
DAU_roll7                   0.944      0.075         0.106   
Ingresos_roll7              0.414      0.147         0.429   

                            Ingresos_Brutos_USD  Unidades_Vendidas  \
DAU                                       0.225              0.215   
Descargas                                 0.128              0.149   
Ingresos_USD                              1.000              0.910   
Ingresos_Brutos_USD                       1.000              0.912   
Unidades_Vendidas                         

 ## 7. Geografía — ¿de dónde viene el dinero de Steam?



 Usamos SteamInGameSales bruto para:

 - Top países por unidades vendidas.

 - Top países por ingresos.



 Esto luego se puede llevar a un mapa en Tableau.

In [31]:
steam_sales_path = os.path.join(RAW_PATH, "SteamInGameSales.csv")
if os.path.exists(steam_sales_path):
    steam_raw = pd.read_csv(steam_sales_path)
    # Aseguramos nombres tal y como los teníamos
    cols = steam_raw.columns.tolist()
    print("Columnas SteamInGameSales RAW:", cols)

    # Top países por unidades
    if "Country" in cols and "Gross Units Sold" in cols:
        top_paises_units = (
            steam_raw.groupby("Country")["Gross Units Sold"]
            .sum()
            .sort_values(ascending=False)
            .head(10)
        )
        print("\n🌍 Top 10 países por unidades vendidas (Steam):")
        print(top_paises_units)

    # Top países por ingresos netos USD
    if "Country" in cols and "Net Steam Sales (USD)" in cols:
        top_paises_ing = (
            steam_raw.groupby("Country")["Net Steam Sales (USD)"]
            .sum()
            .sort_values(ascending=False)
            .head(10)
        )
        print("\n💰 Top 10 países por ingresos netos (Steam, USD):")
        print(top_paises_ing)
else:
    print("⚠ No encuentro SteamInGameSales.csv en RAW_PATH.")


Columnas SteamInGameSales RAW: ['Date', 'Product(ID#)', 'Game', 'Item(ID#)', 'Description', 'Category', 'Country Code', 'Country', 'Region', 'Gross Units Sold', 'Chargeback/Returns', 'Net Units Sold', ' Average Price', 'Currency', 'Gross Steam Sales (USD)', 'Chargeback/Returns (USD)', 'VAT/Tax (USD)', 'Net Steam Sales (USD)']

🌍 Top 10 países por unidades vendidas (Steam):
Country
United States     56879
Canada             7828
United Kingdom     2501
Germany            2405
Australia          1936
France             1574
Austria            1048
Japan               751
Netherlands         722
Sweden              591
Name: Gross Units Sold, dtype: int64

💰 Top 10 países por ingresos netos (Steam, USD):
Country
United States     851939.88
Canada            129424.80
Germany            29602.06
United Kingdom     26239.89
Australia          26128.66
France             20567.40
Austria            11569.99
Netherlands        10410.96
Japan               8718.72
Sweden              6915.30
N

 ## 8. Whales y concentración de gasto (PlayFab)



 Aquí no entra en el FACT, pero sí apoya la historia de:

 - Dependencia de grandes gastadores (whales).

 - Porcentaje de revenue concentrado en el top X% de jugadores.

In [32]:
playfab_path = os.path.join(CLEAN_PATH, "playfab_clean.csv")
if os.path.exists(playfab_path):
    playfab = pd.read_csv(playfab_path)
    if "Revenue" in playfab.columns:
        playfab["Revenue"] = pd.to_numeric(playfab["Revenue"], errors="coerce")
        playfab = playfab[playfab["Revenue"].notna()].copy()

        print("📊 Stats de revenue (PlayFab):")
        desc = playfab["Revenue"].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99]).round(2)
        print(desc)

        p95 = desc["95%"]
        whales = playfab[playfab["Revenue"] >= p95]
        share_jugadores = len(whales) / len(playfab) * 100
        share_revenue = whales["Revenue"].sum() / playfab["Revenue"].sum() * 100

        print(f"\n🐳 Definimos 'whales' como jugadores >= P95 ({p95}).")
        print(f"- % de jugadores que son whales: {share_jugadores:.2f}%")
        print(f"- % del revenue total que aportan: {share_revenue:.2f}%")
    else:
        print("⚠ playfab_clean.csv no tiene columna 'Revenue'.")
else:
    print("⚠ No se ha encontrado playfab_clean.csv. Saltando análisis de whales.")


📊 Stats de revenue (PlayFab):
count    39127.00
mean        14.41
std         12.83
min          0.00
50%         13.50
75%         19.99
90%         25.00
95%         39.99
99%         50.00
max        100.00
Name: Revenue, dtype: float64

🐳 Definimos 'whales' como jugadores >= P95 (39.99).
- % de jugadores que son whales: 6.13%
- % del revenue total que aportan: 22.03%


# 05_INSIGHTS — Proyecto Aurora  
## Análisis interpretativo final

---

## 1. Pregunta principal  
**¿Qué plataforma rinde mejor y qué factores explican la diferencia (usuarios, monetización, estabilidad y dependencia de whales)?**

---

## 2. Resumen ejecutivo  
- Canal B tiene más usuarios (DAU mayor), pero Canal A genera más ingresos.  
- Canal A produce más ingresos diarios (+33%) con menos de la mitad del DAU.  
- Canal A muestra estabilidad; Canal B es muy volátil.  
- Canal B depende de whales; Canal A no.  
- Canal A tiene un modelo de monetización más sostenible.

---

## 3. Evidencias del análisis

### 3.1 KPIs comparativos  
- DAU medio: Canal B > Canal A.  
- Ingresos totales: Canal A > Canal B.  
- Ingresos por día:  
  - Canal A ≈ 1006 USD/día  
  - Canal B ≈ 754 USD/día  
- Descargas: Canal B > Canal A.  

**Conclusión:** Más usuarios no implica más ingresos.

---

### 3.2 Estabilidad del DAU  
- Canal A tiene baja varianza; mantiene una base estable.  
- Canal B fluctúa fuertemente, con picos y caídas.

**Interpretación:** Canal A retiene mejor a los usuarios activos.

---

### 3.3 Evolución temporal  
- Canal A mantiene ingresos y DAU de forma continua.  
- Canal B sufre desgaste en 2025.

**Interpretación:** Canal A es más resiliente; Canal B depende de impulsos puntuales.

---

### 3.4 Correlaciones clave

#### Global  
- DAU no está correlacionado con ingresos.  
- Descargas tampoco explican ingresos.

#### Canal A  
- Ingresos correlacionan con unidades vendidas y ventas brutas.  
- Monetización diversificada y estable.

#### Canal B  
- Ingresos prácticamente idénticos al oro gastado (correlación ~1).  
- ARPU y ARPDAU también se alinean con ingresos.  
- Dependencia clara de compras premium.

**Conclusión:** Canal B es vulnerable; Canal A es más sólido.

---

### 3.5 Distribución del revenue (whales)  
- Los jugadores del percentil 95 generan una parte desproporcionada del revenue en Canal B.  
- Canal A no depende de whales.

---

## 4. Conclusiones finales

### Conclusión 1  
Tener más usuarios no significa rendir mejor.  
Canal A monetiza más con menos DAU.

### Conclusión 2  
Canal A es estable; Canal B es volátil.

### Conclusión 3  
El modelo de monetización de Canal A es sostenible; el de Canal B depende de whales.

### Conclusión 4  
A lo largo del tiempo, Canal A mantiene crecimiento y resiliencia.

---

## Conclusión final del proyecto  
**La plataforma con mejor rendimiento global es Canal A**, porque:  
1. Genera más ingresos totales y diarios.  
2. Monetiza mejor por usuario.  
3. Depende menos de whales.  
4. Tiene mayor estabilidad en DAU e ingresos.  
5. Mantiene consistencia a largo plazo.


